In [ ]:
!pip install tika

  Created wheel for tika: filename=tika-1.24-cp36-none-any.whl size=32885 sha256=38b6821fb7ad4103b017cb493e36ac0c948a6beddb47fb9e2b8341e125192de7
  Stored in directory: /root/.cache/pip/wheels/73/9c/f5/0b1b738442fc2a2862bef95b908b374f8e80215550fb2a8975
Successfully built tika


In [ ]:
import pandas as pd
import numpy as np
import os
import tika
from tika import parser
#from pdf_scraper import PdfScraper

In [ ]:
#colab stuff
from google.colab import files

#testing in colab code
from google.colab import drive
drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive'
os.chdir('gdrive/My Drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
os.chdir('colabdata')

In [ ]:
pdf_list = os.listdir()[:3]

In [ ]:
pdf_scraper = PdfScraper()
df = pdf_scraper.scrape(pdf_list)

In [ ]:
df.to_csv('pdf_extracts.csv')

In [ ]:
#colab
files.download('pdf_extracts.csv')

In [ ]:
class PdfScraper:

    def clean(self, lines):
        #clean
        bad = ['Microsoft Word', 'Resource Report: Food Programs', 'Nutrition, Food Security, Food', 'Report Date', 'Created by Zoe Arms', 'Financial Resources', 'DISCLAIMER']
        clean_lines = []
        for i in range(len(lines)):
            good_line = True
            for b in bad:
                if b in lines[i]:
                    good_line = False
                elif i > 1 and b == 'DISCLAIMER' and (b in lines[i-1] or b in lines[i-2]):
                    good_line = False
            if lines[i] == '' or lines[i] == ' ' or lines[i] == '  ' or lines[i] == '     ':
                good_line = False
            if good_line:
                clean_lines.append(lines[i])
        return clean_lines
    
    def get_entries(self, clean_lines):
        entries = []
        entry = []
        for line in clean_lines:
            if 'No Cost Share' in line:
                entries.append(entry)
                entry = []
            else:
                entry.append(line)
        return entries
    
    def get_data(self, entries):
        data = []
        search_terms = ['Total Funding:', 'Dept/Agency/Org:','Description:','Website:','Eligibility:']
        for entry in entries:
            record = []
            x = ''
            m = 0
            for line in entry:
                if 'Funding not id' in line:
                    record.append(x)
                    x = ''
                    m += 1
                if m < len(search_terms) and search_terms[m] in line:
                    record.append(x)
                    x = line[len(search_terms[m]):]
                    m += 1
                else:
                    x += line
            record.append(x)
            data.append(record)
        return data
    
    def make_df(self, data,fp):
        sponsors = []
        sponsor_types = []
        org_urls = []
        program_urls = []
        text_descrs = []
        eligibities = []
        program_names = []
        total_funds = []
        deadlines = []
        notes = []
        org_cities = []
        org_states = []
        sources = []
        source_last_updates = []
        content_types  = []
        prog_target_vals = []
        cols= ['sponsor', 'sponsor_type', 'org_url', 'program_url', 'text_descr','eligibility', 'program_name', 'total_funds', 'deadline', 'notes', 'org_city', 'org_state', 'source', 'source_last_update', 'content_type', 'prog_target']
        target_terms = {'Small Business': 'small_biz', 'Nonprofit': 'nonprofit', 'Large Business': 'all_biz', 'Individuals': 'ind'}
        sponsor_terms = {'Federal':'public: federal', 'State': 'public: state', 'Local': 'public: city, public: state', 'Nonprofit': 'private: nonprofit', 'Private': 'private: public_company, private: private_company, private: foundation'}
        a = 0
        b = 1
        c = 2
        d = 3
        e = 4
        f = 5
        for entry in data:
            sources.append(fp)
            source_last_updates.append('')
            content_types.append('other')
            x = entry[a].split('Deadline: ')
            program_names.append(x[0])
            total_funds.append(entry[b])
            try:
                sponsors.append(entry[c])
            except:
                sponsors.append('')
            try:
                text_descrs.append(entry[d])
            except:
                text_descrs.append('')
            org_cities.append('')
            org_states.append('')
            try:
                deadlines.append(x[1])
            except:
                deadlines.append('')
            try:
                program_urls.append(entry[e])
                osplit = entry[e].split('.gov')
                if len(osplit) > 1:
                    org_url = osplit[0] + '.gov'
                    org_urls.append(org_url)
                else:
                    org_urls.append(entry[e])
            except:
                program_urls.append('')
                org_urls.append('')
            try:
                y = entry[f].split('Notes: ')
                eligibities.append(y[0])
                terms = ''
                spons = ''
                for term in target_terms.keys():
                    if term in entry[f]:
                        terms += target_terms[term] + ', '
                for spon in sponsor_terms.keys():
                    if spon in entry[f]:
                        spons += sponsor_terms[spon] + ', '
                if terms == '':
                    terms = 'other'
                else:
                    terms = terms[:-2]
                prog_target_vals.append(terms)
                if spons == '':
                    spons = 'unknown'
                else:
                    spons = spons[:-2]
                sponsor_types.append(spons)
                try:
                    notes.append(y[1])
                except:
                    notes.append('')
            except:
                eligibities.append('')
                notes.append('')
                prog_target_vals.append('other')
                sponsor_types.append('unknown')
        df = pd.DataFrame(list(zip(sponsors,sponsor_types,org_urls,program_urls,text_descrs,eligibities,program_names,total_funds,deadlines,notes,org_cities,org_states,sources,source_last_updates,content_types,prog_target_vals)),columns=cols)
        return df
    
    #call this method with list of paths to pdfs as input, output is a dataframe that can be appended to dfs from other scrapers
    def scrape(self, pdf_list):
        df = pd.DataFrame()
        for fp in pdf_list:
            file_data = parser.from_file(fp)
            text = file_data['content']
            lines = text.split('\n')
            clean_lines = self.clean(lines)
            entries = self.get_entries(clean_lines)
            data = self.get_data(entries)
            df = df.append(self.make_df(data, fp),ignore_index=True)
        return df